In [1]:
import sys
import re
from glob import glob
import os
from itertools import groupby
import pickle
import bs4
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import json
# import nltk
# nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

In [2]:
recipes={}
inv_idx={}
ing_list=[]
with open('epi_r.csv', newline='',encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in csvreader:
        if not inv_idx:
            ing_list=row
            for idx in range(1,len(row)):
                inv_idx[row[idx]]=[]
        else:
            recipe=row[0].rstrip()
            recipes[recipe]={}
            recipes[recipe]['nutrition']={}
            recipes[recipe]['features']={}
            for idx in range(2,len(row)):
                recipe_name=row[0]
                if row[idx]!='':
                    if int(row[idx])>0:
                        #add recipe name to inv_idx
                        if idx>5:
                            inv_idx[ing_list[idx]].append(recipe_name.rstrip())
                            #add ingredients recipe dictionary
                            recipes[recipe]['features'][ing_list[idx]]=int(row[idx])
                        else: 
                            recipes[recipe]['nutrition'][ing_list[idx]]=int(row[idx])


    #print(recipes.keys())
    # print('Lentil Apple and Turkey Wrap',": ",recipes['Lentil Apple and Turkey Wrap'])
            
    # print(inv_idx['apple'])

In [3]:
class IngredientIndex:
    ''' Class representing an inverted index search tool'''
    
    def __init__(self):
        
        self.inv_idx = {}

        with open('full_format_recipes.json', newline='', encoding='utf-8') as f:
            recipes_list = json.load(f)
            for recipe_dict in recipes_list:
                if 'title' in recipe_dict:
                    title = recipe_dict['title'].rstrip().replace(',','')
                    if title in recipes:
                        recipes[title]['ingredients'] = set()
                        for ingredient_str in recipe_dict['ingredients']:
                            for ingredient in ingredient_str.split():
                                ingredient = lemmatizer.lemmatize(ingredient.lower().replace(',',''))
                                recipes[title]['ingredients'].add(ingredient)
                                if ingredient not in self.inv_idx:
                                    self.inv_idx[ingredient] = set()
                                self.inv_idx[ingredient].add(title)
    
    def __getitem__(self, item):
        return self.inv_idx[lemmatizer.lemmatize(item)]
    
    # Returns set of recipes such that every ingredient in ingredient_list is included in each recipe
    def intersect(self, ingredient_list):
        
        result = -1
        
        for ingredient in ingredient_list:
            if result == -1:
                result = self[ingredient]
            else:
                result = result.intersection(self[ingredient])
        
        if result == -1:
            result = set()
        
        return result

    # Returns set of recipes such that at least one ingredient in ingredient_list is included in each recipe
    def union(self, ingredient_list):
        
        result = set()
        
        for ingredient in ingredient_list:
            result = result.union(self[ingredient])
                
        return result
    
    # Returns set of recipes such that no ingredient in ingredient_list is included in each recipe
    def exclude(self, ingredient_list):
        
        bad_recipes = self.union(ingredient_list)
        
        result = set()
        
        for recipe in recipes:
            if recipe not in bad_recipes:
                result.add(recipe)
        
        return result
    
    # Returns set of recipes such that every ingredient in good_ingredient_list is included in each recipe
    # and every ingredient in bad_ingredient_list is not included in each recipe
    def search(self, good_ingredient_list, bad_ingredient_list):
        
        return self.intersect(good_ingredient_list).intersection(self.exclude(bad_ingredient_list))
    
temp = IngredientIndex()

In [4]:
print(len(temp['apple']))

print(len(temp.exclude([])))

print(len(temp.intersect(['apple', 'chicken'])))

print('Cider-Braised Chicken' in temp['lemon'])

print(len(temp.exclude(['lemon', 'apple', 'chicken'])))

print(temp.search(['apple', 'chicken', 'sage'], ['turkey', 'flour']))

# No way to sort by flavor (i.e. salty, sweet)
# No way to filter by allergies or dietary restrictions that encompass more than a single ingredient
# Doesn't correctly deal with ingredients with multi-word names (i.e. black pepper)

731
14081
121
False
8649
{'Sausage Cranberry and Pecan Stuffing', 'Sourdough Italian Sausage and Chestnut Stuffing', 'Braised Chicken with Apples and Sage', 'Apple and Sausage Stuffing', 'Cider-Braised Chicken', 'Apple Sausage and Parsnip Stuffing with Fresh Sage', 'Sweet Potato and Apple Dressing', 'Chestnut Bacon Dried Apple and Corn Bread Stuffing', 'New England Sausage Stuffing with Maple Corn Bread', 'Butternut Squash Soup with Cider Cream', 'Apple Celery and Smoked Ham Stuffing', 'Grilled Chicken Sandwiches with Sage Pesto and Apples', 'Prune Apple and Chestnut Bread Pudding', 'Smoked-Cheddar-Stuffed Chicken with Green Apple Slaw'}


In [5]:
    i = 0
    for thing in inv_idx:
        print(thing)
        i += 1
        if i == 10:
            break

rating
calories
protein
fat
sodium
#cakeweek
#wasteless
22-minute meals
3-ingredient recipes
30 days of groceries


In [6]:
print('Lentil Apple and Turkey Wrap ',": ",recipes['Lentil Apple and Turkey Wrap'])
print(len(inv_idx['apple']))

Lentil Apple and Turkey Wrap  :  {'nutrition': {'calories': 426, 'protein': 30, 'fat': 7, 'sodium': 559}, 'features': {'apple': 1, 'bean': 1, 'cookie': 1, 'fruit': 1, 'kid-friendly': 1, 'lentil': 1, 'lettuce': 1, 'san francisco': 1, 'tomatillo': 1, 'vegan': 1, 'snack': 1}, 'ingredients': {'head', '(12-inch)', '4', 'chicken', 'cut', 'olive', 'stock', '1/2', 'juice', 'thinly', 'thyme', 'peeled', 'extra-virgin', 'squeezed', '3', '3/4', 'chopped', 'salt', 'lentil', 'breast', 'lavash', 'ground', 'whole-wheat', 'kosher', 'cup', 'flour', 'pepper', 'to', 'carrot', 'crosswise', 'or', 'low-sodium', 'sprig', 'seeded', 'green', 'tablespoon', 'sheet', 'tomato', 'large', 'stalk', 'black', 'pound', 'taste', 'lemon', 'turkey', '1', 'sliced', '6', 'medium', 'lettuce', 'brown', '2', 'fresh', 'celery', 'dried', 'and', 'small', 'half', 'bibb', 'tortilla', 'french', 'fuji', 'oil', 'in', 'vegetable', 'apple', 'freshly', 'teaspoon', 'cored', 'diced'}}
543


In [7]:
for item in inv_idx['apple']:
    if item not in temp['apple']:
        print(item)

"""Seder Plate"" Salad "
Cranberry and Celery Relish
The Widow's Touch
Mulled Cider
Applesauce Spice Muffins
"Apple ""Pizza"" "
Applejack Toddy
Ladies' Mile Applesauce Raisin Cake Diamond
Honey-Caramel Tart with Apricots and Almonds
"Apple ""Flag"" Tart "
Apple Pie
Four Spirits Eggnog
Applesauce with Butter Pecan Crumbs
Spiced Cider Cup
Chicken-Apple Sausage and Fresh Fennel Stuffing
Raspberry and Rosé Petal Punch
Curried Pork Chops with Brandied Peach and Crabapple Chutney
"Gala Apple ""Newtons"" with Honey Cream "
Winter Salad with Black Radish Apple and Escarole
Creamy Apple Horseradish Sauce
Russian Walnut-Cherry Latkes with Cherry-Apple Sauce
"Brioche ""Bread Pudding"" with Caramelized Apples "
"Smoky Grilled Pork Steaks with ""Magic Dust"" "


In [8]:
for item in temp['apple']:
    print(item)
#     if item not in inv_idx['apple']:
#         print(item)

Cabbage and Apple Slaw with Butter-Toasted Pecans
Tea-Flavored Tarte Tatin (Tarte Tatin au Thé)
Apple Horseradish Aspic with Fennel
Spiced Streusel Apple Pie
Rum-Raisin Apple Cake
Red Wine Barbecue Sauce
Caramel Apple Pastis
Chipotle Sauce
Tender Pie Crust
Breakfast Couscous with Dried-fruit compote
Apple Cobblers
Cider-Glazed Roasted Root Vegetables
Cinnamon Roulade with Apple Raisin Filling
Fennel and Apple Salad with Cider Vinaigrette
Apple Cider Onion and Raisin Chutney
Ginger Baked Apples
Roasted Red Pepper and Tomato Sauce
Baked Chicken with Barbecue Sauce
Chicken Breasts with Chive and Mustard Sauce
Pork Loin with Apples Prunes and Mustard Cream Sauce
Smothered Grilled Pork Chops
Celery and Apple Salad Dijon
Fennel Watercress and Endive Salad with Toasted Pine Nuts
Crown Roast of Pork with Apple Stuffing
Mojo Pork Tacos With Sweet Apple Slaw
Blue Cheese Coleslaw
Onion Soup with Apple Cider
Coleslaw with Apple and Yogurt Dressing
Roast Vermont Turkey with Giblet Gravy and Sausage